In [4]:
import pandas as pd
import requests

In [5]:
# Import library yang diperlukan
from flask_mysqldb import MySQL
from flask import Flask
import pandas as pd

# Inisialisasi aplikasi Flask dan koneksi MySQL
app = Flask(__name__)

# Konfigurasi database MySQL
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = ''  # Ubah sesuai password MySQL Anda
app.config['MYSQL_DB'] = 'penjadwalan'  # Nama database Anda

# Inisialisasi MySQL
mysql = MySQL(app)

# Membuat konteks aplikasi Flask untuk akses database
with app.app_context():
    # Query untuk mengambil data kelas
    query = """
        SELECT 
            kelas_db.id_kelas, 
            matakuliah_db.matakuliah,
            kelas_db.skala,
            kelas_db.kelas,
            GROUP_CONCAT(DISTINCT dosen_db.dosen ORDER BY kelas_dosen.urutan_dosen SEPARATOR '/n ') AS dosen,
            GROUP_CONCAT(DISTINCT rooms_db.rooms ORDER BY kelas_rooms.urutan_rooms SEPARATOR ', ') AS rooms,
            kelas_db.kapasitas,
            kelas_db.createdAt,
            matakuliah_db.jenjang,
            matakuliah_db.wp,
            GROUP_CONCAT(DISTINCT semester_db.semester ORDER BY semester_db.semester SEPARATOR ', ') AS semester_list,
            matakuliah_db.durasi
        FROM 
            kelas_db
        JOIN 
            matakuliah_db ON kelas_db.matkul_id = matakuliah_db.id_matkul
        LEFT JOIN 
            kelas_dosen ON kelas_db.id_kelas = kelas_dosen.kelas_id
        LEFT JOIN 
            dosen_db ON kelas_dosen.dosen_id = dosen_db.id_dosen
        LEFT JOIN 
            kelas_rooms ON kelas_db.id_kelas = kelas_rooms.kelas_id
        LEFT JOIN 
            rooms_db ON kelas_rooms.rooms_id = rooms_db.id_rooms
        LEFT JOIN 
            semester_matakuliah ON matakuliah_db.id_matkul = semester_matakuliah.matkul_id
        LEFT JOIN 
            semester_db ON semester_matakuliah.semester_id = semester_db.id_semester
        GROUP BY 
            kelas_db.id_kelas, 
            matakuliah_db.matakuliah, 
            kelas_db.skala, 
            kelas_db.kelas, 
            kelas_db.kapasitas, 
            kelas_db.createdAt, 
            matakuliah_db.jenjang,
            matakuliah_db.wp,
            matakuliah_db.durasi
    """

    # Menjalankan query dan mendapatkan data
    cur = mysql.connection.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    cur.close()

    # Memproses data yang diambil dan membuat DataFrame
    kelas_data = [
        {
            'id_kelas': row[0],
            'matakuliah': row[1],
            'skala': row[2],
            'kelas': row[3],
            'dosen': row[4].split('/n ') if row[4] else [],
            'rooms': (row[5].split(', ') if row[5] else []) + ['alternatif'],
            'kapasitas': row[6],
            'createdAt': row[7],
            'jenjang': row[8],
            'wp': row[9],
            'semester': [int(x) for x in row[10].split(', ')] if row[10] else [],
            'durasi': row[11],
        }
        for row in rows
    ]

    # Membuat DataFrame untuk menampilkan data
    df = pd.DataFrame(kelas_data)

    # Menampilkan DataFrame
    # display(df)


In [6]:
# Import library yang diperlukan
from flask_mysqldb import MySQL
from flask import Flask
import pandas as pd

# Inisialisasi aplikasi Flask dan koneksi MySQL
app = Flask(__name__)

# Konfigurasi database MySQL
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = ''  # Ubah sesuai password MySQL Anda
app.config['MYSQL_DB'] = 'penjadwalan'  # Nama database Anda

# Inisialisasi MySQL
mysql = MySQL(app)

# Membuat konteks aplikasi Flask untuk akses database
with app.app_context():
    # Query untuk mengambil data ruangan
    query = """
        SELECT 
            ruangan_db.id_ruangan, 
            ruangan_db.name, 
            ruangan_db.kapasitas, 
            GROUP_CONCAT(jenis_db.jenis ORDER BY jenis_ruangan.jenis_id SEPARATOR ', ') AS jenis_list,
            ruangan_db.createdAt
        FROM 
            ruangan_db
        LEFT JOIN 
            jenis_ruangan ON ruangan_db.id_ruangan = jenis_ruangan.ruangan_id
        LEFT JOIN 
            jenis_db ON jenis_ruangan.jenis_id = jenis_db.id_jenis
        GROUP BY 
            ruangan_db.id_ruangan, ruangan_db.name, ruangan_db.kapasitas, ruangan_db.createdAt
    """

    # Menjalankan query dan mendapatkan data
    cur = mysql.connection.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    cur.close()

    # Memproses data yang diambil dan membuat DataFrame
    ruangan_data = [
        {
            'id_ruangan': row[0],
            'name': row[1],
            'kapasitas': row[2],
            'jenis': row[3].split(', ') if row[3] else [],  # Memecah jenis ruangan yang dipisahkan koma
            'createdAt': row[4],
        }
        for row in rows
    ]

    # Membuat DataFrame untuk menampilkan data
    df = pd.DataFrame(ruangan_data)

    # Menampilkan DataFrame
    # display(df)


In [7]:
hari_data = [
    "Senin", 
    "Selasa", 
    "Rabu", 
    "Kamis", 
    "Jumat"
]
jam_data = [
    "07:50 - 08:40", "08:40 - 09:30",
    "09:30 - 10:20", "10:20 - 11:10", 
    "11:10 - 12:00", "12:00 - 13:00",
    "13:00 - 13:50", "13:50 - 14:40", 
    "14:40 - 15:30", "15:30 - 16:20", 
    "16:20 - 17:10"
]

In [8]:
jadwal_s1 = []
scheduled_classes = []  
tabrakan_data = []
failed_classes = []

def are_slots_connected(jam_slots):
    # Periksa apakah slot mengandung jam istirahat
    if "12:00 - 13:00" in jam_slots:
        return False

    # Periksa konektivitas antar slot
    for i in range(len(jam_slots) - 1):
        if jam_slots[i] == "11:10 - 12:00" and jam_slots[i + 1] == "13:00 - 13:50":
            return False

    return True

# Fungsi untuk mengecek tabrakan jadwal dengan rentang minggu
def is_schedule_valid(kelas, dosen, ruangan, hari, jam_slots, minggu):
    # Definisikan rentang minggu dalam format numerik untuk perbandingan
    minggu_map = {
        "1-6": (1, 6),
        "7-11": (7, 11),
        "12-16": (12, 16),
        "1-8": (1, 8),
        "9-16": (9, 16),
        "1-16": (1, 16)
    }

    start_minggu, end_minggu = minggu_map[minggu]

    # Periksa apakah ada tabrakan dengan jadwal lain yang sudah terpasang
    for entry in jadwal_s1:
        entry_start_minggu, entry_end_minggu = minggu_map[entry['minggu']]

        # Cek apakah ada tabrakan minggu dan jadwal (ruangan atau dosen)
        if entry['ruangan'] == ruangan and entry['hari'] == hari and \
        entry_start_minggu <= end_minggu and entry_end_minggu >= start_minggu and \
        any(jam in entry['jam'] for jam in jam_slots):
            return False

        if entry['dosen'] == dosen and entry['hari'] == hari and \
        entry_start_minggu <= end_minggu and entry_end_minggu >= start_minggu and \
        any(jam in entry['jam'] for jam in jam_slots):
            return False
    return True

def is_odd(number):
    return number % 2 != 0

In [9]:
# filter S1, Nasional, semester ganjil
def filter_and_sort_kelas(kelas_data, w_p, jenjang, skala, target_ganjil=True):
    # Filter kelas berdasarkan kondisi yang diberikan
    filtered_kelas = [
        kelas for kelas in kelas_data
        if kelas.get("wp") == w_p and
        kelas.get("jenjang") == jenjang and
        kelas.get("skala") == skala and
        (
            # Jika target adalah ganjil, cek apakah ada angka ganjil di semester
            (target_ganjil and any(is_odd(s) for s in kelas.get("semester", []))) or
            # Jika target adalah genap, cek apakah ada angka genap di semester
            (not target_ganjil and any(not is_odd(s) for s in kelas.get("semester", [])))
        )
    ]

    # Urutkan berdasarkan kapasitas dan durasi
    return sorted(
        filtered_kelas,
        key=lambda x: (x["kapasitas"], x["durasi"]),
        reverse=True
    )
filtered_kelas_w = filter_and_sort_kelas(kelas_data, "W", "S1", "Nasional", target_ganjil=True)  # Untuk semester ganjil
filtered_kelas_p = filter_and_sort_kelas(kelas_data, "P", "S1", "Nasional", target_ganjil=True)  # Untuk semester genap
filtered_kelas_data = filtered_kelas_w + filtered_kelas_p

def filter_ruangan(kelas, ruangan_data):
    # List prioritas rooms dari kelas
    rooms_prioritas = kelas.get('rooms', [])
    kapasitas_kelas = kelas['kapasitas']

    # Inisialisasi ruangan terpilih kosong
    ruangan_terpilih = []

    # Iterasi berdasarkan prioritas room
    for room_prioritas in rooms_prioritas:
        # Filter ruangan berdasarkan jenis dan kapasitas
        ruangan_cocok = [
            ruangan for ruangan in ruangan_data
            if room_prioritas in ruangan.get('jenis', []) and ruangan['kapasitas'] >= kapasitas_kelas
        ]

        # Jika ditemukan ruangan cocok, tambahkan ke ruangan terpilih dan keluar dari loop
        if ruangan_cocok:
            ruangan_terpilih.extend(ruangan_cocok)
            break

    # Jika tidak ada ruangan sesuai prioritas pertama, gunakan alternatif
    if not ruangan_terpilih:
        ruangan_terpilih = [
            ruangan for ruangan in ruangan_data
            if 'alternatif' in ruangan.get('jenis', []) and ruangan['kapasitas'] >= kapasitas_kelas
        ]

    # Urutkan ruangan berdasarkan kapasitas (ascending)
    ruangan_terpilih = sorted(ruangan_terpilih, key=lambda x: x['kapasitas'])

    return ruangan_terpilih

def schedule_nasional_s1_ganjil_classes():
    for kelas in filtered_kelas_data:
        if 'ruangan' in kelas and 'jam' in kelas:
            continue

        is_scheduled = False
        for hari in hari_data:
            if is_scheduled:  # Stop pencarian jika sudah dijadwalkan
                break

            # Filter ruangan yang memungkinkan
            ruangan_terpilih = filter_ruangan(kelas, ruangan_data)
            if not ruangan_terpilih:
                continue

            for ruangan in ruangan_terpilih:
                if is_scheduled:
                    break

                jam_index = 0
                while jam_index <= len(jam_data) - kelas["durasi"]:
                    jam_slots = jam_data[jam_index:jam_index + kelas["durasi"]]

                    # Gabungkan validasi konektivitas dan jam istirahat
                    if not are_slots_connected(jam_slots):
                        jam_index += 1
                        continue

                    # Pembagian minggu berdasarkan jumlah dosen
                    dosen_minggu_map = []
                    if len(kelas["dosen"]) == 1:
                        dosen_minggu_map = [(kelas["dosen"][0], "1-16")]
                    elif len(kelas["dosen"]) == 2:
                        dosen_minggu_map = [
                            (kelas["dosen"][0], "1-8"),
                            (kelas["dosen"][1], "9-16")
                        ]
                    elif len(kelas["dosen"]) == 3:
                        dosen_minggu_map = [
                            (kelas["dosen"][0], "1-6"),
                            (kelas["dosen"][1], "7-11"),
                            (kelas["dosen"][2], "12-16")
                        ]

                    # Validasi setiap pasangan dosen dan minggu
                    conflict_found = False
                    for dosen, minggu in dosen_minggu_map:
                        if not is_schedule_valid(kelas, dosen, ruangan["name"], hari, jam_slots, minggu):
                            conflict_found = True
                            break

                    if conflict_found:
                        jam_index += 1
                        continue

                    # Jadwalkan kelas jika valid
                    for dosen, minggu in dosen_minggu_map:
                        jadwal_s1.append({
                            "kelas": kelas["kelas"],
                            "matakuliah": kelas["matakuliah"],
                            "dosen": dosen,
                            "ruangan": ruangan["name"],
                            "w/p": kelas["wp"],
                            "semester": kelas["semester"],
                            "hari": hari,
                            "jam": jam_slots,
                            "minggu": minggu
                        })

                    # Tandai kelas sebagai dijadwalkan
                    kelas['ruangan'], kelas['jam'], kelas['hari'] = ruangan, jam_slots, hari
                    is_scheduled = True
                    scheduled_classes.append(kelas)
                    break

        if not is_scheduled:
            failed_classes.append(kelas)
            print(f">>>>> Warning: Kelas {kelas['kelas']} tidak dapat dijadwalkan di ruangan dengan room: {kelas['rooms']}")

# Menjalankan penjadwalan
schedule_nasional_s1_ganjil_classes()

In [10]:
if not jadwal_s1:  # Periksa jika jadwal kosong
    print("Tidak ada data kelas yang didapatkan")
else:
    # Konversi jadwal ke DataFrame
    df_jadwal_normal = pd.DataFrame(jadwal_s1)
    df_jadwal_normal['jam'] = df_jadwal_normal['jam'].apply(lambda x: ', '.join(map(str, x)))

    df_jadwal_sorted = df_jadwal_normal.sort_values(by=['hari', 'jam'])
    # display(df_jadwal_sorted)